In [1]:
import tensorflow as tf
import glob
import imageio
import matplotlib as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time

from IPython import display

Num GPUs Available:  1


In [ ]:
(train_images, train_labels),(_,_) = tf.keras.datasets.cifar10.load_data() # (-,-)) we don't need test data

train_images = train_images.reshape(train_images.shape[0], 32, 32, 3).astype('float32') 
train_images = (train_images - 127.5)/ 127.5 # Normalize the images to [-1, 1]

BUFFER_SIZE = 50000
BATCH_SIZE = 256

# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(4*4*256, use_bias=False, input_shape=(100,))) # 4*4*256 = 4096
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(layers.Reshape((4, 4, 256))) # 4*4*256 = 4096
    assert model.output_shape == (None, 4, 4, 256) # Note: None is the batch size

    model.add(tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False)) # 8*8*128 = 8192
    assert model.output_shape == (None, 8, 8, 128)
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False)) # 16*16*64 = 16384
    assert model.output_shape == (None, 16, 16, 64)
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')) # 32*32*3 = 3072
    assert model.output_shape == (None, 32, 32, 3)

    return model


#use generator (untrained)) to create an image
generator = make_generator_model()
noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)
plt.imshow(generated_image[0, :, :, 0], cmap='gray')

## Discriminator

In [2]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[32, 32, 3])) # 16*16*64 = 16384
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same')) # 8*8*128 = 8192 # 5,5 = kernel size
    model.add(tf.keras.layers.LeakyReLU()) 
    model.add(tf.keras.layers.Dropout(0.3)) 

    model.add(layers.Flatten()) # 8*8*128 = 8192 
    model.add(tf.keras.layers.Dense(1))

    return model

#use the untrained discriminator to classify the generated images as real or fake. The model will be trained to output positive values for real images, and negative values for fake images.
discriminator = make_discriminator_model()
decision = discriminator(generated_image)

NameError: name 'generated_image' is not defined

In [3]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,discriminator_optimizer=discriminator_optimizer,generator=generator,discriminator=discriminator)

NameError: name 'generator' is not defined

The training loop will begin with the *generator* receiving a random seed as input. That seed is used to produce an image.
The *discriminator* is then used to classify *real images*(from the training set) and *fakes* (from the generator).  
The _loss_ is calculated for each and the gradients are used to update the generator and discriminator.

In [ ]:
EPOCHS = 50 # 50 epochs to train the model
noise_dim = 100 # 100 random numbers to generate an image
num_examples_to_generate = 16 #
seed = tf.random.normal([num_examples_to_generate, noise_dim]) # seed for the generator that we will use to visualize progress in the animated GIF

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim]) # generate noise
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape: # use gradient tape to record gradients
        generated_images = generator(noise, training=True) # generate fake images

        real_output = discriminator(images, training=True) # train discriminator on real images
        fake_output = discriminator(generated_images, training=True) # train discriminator on fake images

        gen_loss = generator_loss(fake_output) # calculate generator loss
        disc_loss = discriminator_loss(real_output, fake_output) # calculate discriminator loss

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables) 

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))  

In [6]:
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for image_batch in dataset:
            train_step(image_batch)

        #produce images for the GIF
        display.clear_output(wait=True)
        generate_and_save_images(generator,epoch + 1,seed)

        #save the model every 15 epochs
        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)
        print('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

        #generate after the final epoch
        display.clear_output(wait=True)
        generate_and_save_images(generator,epochs,seed)


def generate_and_save_images(model,epoch,test_input):
    #training is set to False so all layers run in inference mode (batchnorm)
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4,4))

    for i in range(predictions.shape[0]):
        plt.subplot(4,4,i+1)
        plt.imshow(predictions[i, :, :, :] * 127.5 + 127.5)
        plt.axis('off')
    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()        

In [ ]:
train(train_dataset, EPOCHS)


In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
def display_image(epoch_no):
    return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

anim_file = 'dcgan.gif' #name of the file
with imageio.get_writer(anim_file, mode='I') as writer: #get_writer() returns a writer object that will call append_data() to add a new frame to the GIF file.
    filenames = glob.glob('image*.png') #glob.glob() returns a list of files that match the given pattern
    filenames = sorted(filenames) 
    for filename in filenames:
        image = imageio.imread(filename) #imageio.imread() reads an image from the file
        writer.append_data(image) #append_data() adds a new frame to the GIF file
    image = imageio.imread(filename)
    writer = append_data(image) #append_data() adds a new frame to the GIF file
    

In [ ]:
import tensorflow_docs.vis.embed as embed
embed.embed_file(anim_file)